In [1]:
# Import a bunch of libraries.

import re
import time
import gc

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

%matplotlib inline


In [2]:
debug = False # default: False. if True, uses existing mini_initial.csv file so this runs fast
save_data = True # default: True. if False, will not save, which takes the most time


In [3]:
#
# This loads the data and forces certain types
#
def load_data(filename):
    
    dtypes = {
        'MachineIdentifier':                                    'str',
        'ProductName':                                          'str',
        'EngineVersion':                                        'str',
        'AppVersion':                                           'str',
        'AvSigVersion':                                         'str',
        'IsBeta':                                               'int8',
        'RtpStateBitfield':                                     'float64',
        'IsSxsPassiveMode':                                     'int8',
        'DefaultBrowsersIdentifier':                            'float32',
        'AVProductStatesIdentifier':                            'float32',
        'AVProductsInstalled':                                  'float16',
        'AVProductsEnabled':                                    'float16',
        'HasTpm':                                               'int8',
        'CountryIdentifier':                                    'int16',
        'CityIdentifier':                                       'float32',
        'OrganizationIdentifier':                               'float16',
        'GeoNameIdentifier':                                    'float16',
        'LocaleEnglishNameIdentifier':                          'int16',
        'Platform':                                             'category',
        'Processor':                                            'category',
        'OsVer':                                                'category',
        'OsBuild':                                              'int16',
        'OsSuite':                                              'int16',
        'OsPlatformSubRelease':                                 'category',
        'OsBuildLab':                                           'category',
        'SkuEdition':                                           'category',
        'IsProtected':                                          'float16',
        'AutoSampleOptIn':                                      'int8',
        'PuaMode':                                              'category',
        'SMode':                                                'float16',
        'IeVerIdentifier':                                      'float16',
        'SmartScreen':                                          'str',
        'Firewall':                                             'float16',
        'UacLuaenable':                                         'float64', 
        'Census_MDC2FormFactor':                                'category',
        'Census_DeviceFamily':                                  'category',
        'Census_OEMNameIdentifier':                             'float32', 
        'Census_OEMModelIdentifier':                            'float32',
        'Census_ProcessorCoreCount':                            'float16',
        'Census_ProcessorManufacturerIdentifier':               'float16',
        'Census_ProcessorModelIdentifier':                      'float32', 
        'Census_ProcessorClass':                                'category',
        'Census_PrimaryDiskTotalCapacity':                      'float64', 
        'Census_PrimaryDiskTypeName':                           'category',
        'Census_SystemVolumeTotalCapacity':                     'float64', 
        'Census_HasOpticalDiskDrive':                           'int8',
        'Census_TotalPhysicalRAM':                              'float32',
        'Census_ChassisTypeName':                               'str',
        'Census_InternalPrimaryDiagonalDisplaySizeInInches':    'float32', 
        'Census_InternalPrimaryDisplayResolutionHorizontal':    'float32', 
        'Census_InternalPrimaryDisplayResolutionVertical':      'float32', 
        'Census_PowerPlatformRoleName':                         'category',
        'Census_InternalBatteryType':                           'str',
        'Census_InternalBatteryNumberOfCharges':                'float64', 
        'Census_OSVersion':                                     'category',
        'Census_OSArchitecture':                                'category',
        'Census_OSBranch':                                      'category',
        'Census_OSBuildNumber':                                 'int16',
        'Census_OSBuildRevision':                               'int32',
        'Census_OSEdition':                                     'str',
        'Census_OSSkuName':                                     'category',
        'Census_OSInstallTypeName':                             'category',
        'Census_OSInstallLanguageIdentifier':                   'float16',
        'Census_OSUILocaleIdentifier':                          'int16',
        'Census_OSWUAutoUpdateOptionsName':                     'category',
        'Census_IsPortableOperatingSystem':                     'int8',
        'Census_GenuineStateName':                              'category',
        'Census_ActivationChannel':                             'category',
        'Census_IsFlightingInternal':                           'float16',
        'Census_IsFlightsDisabled':                             'float16',
        'Census_FlightRing':                                    'category',
        'Census_ThresholdOptIn':                                'float16',
        'Census_FirmwareManufacturerIdentifier':                'float16',
        'Census_FirmwareVersionIdentifier':                     'float32',
        'Census_IsSecureBootEnabled':                           'int8',
        'Census_IsWIMBootEnabled':                              'float16',
        'Census_IsVirtualDevice':                               'float16',
        'Census_IsTouchEnabled':                                'int8',
        'Census_IsPenCapable':                                  'int8',
        'Census_IsAlwaysOnAlwaysConnectedCapable':              'float16',
        'Wdft_IsGamer':                                         'float16',
        'Wdft_RegionIdentifier':                                'float16',
        'HasDetections':                                        'int8'
        }

    df = pd.read_csv("data/"+filename+".csv", dtype=dtypes, engine='c')
    return df

In [4]:
def clean_data(df):
        
    # drop machine identifiers, so models don't train on them
#     if "MachineIdentifier" in df.columns:
#         df = df.drop(columns=["MachineIdentifier"])
    
    #
    # make all strings lower case
    # get rid of hex char codes, keep the actual code number
    #
    
    char_treatment = [
        'AvSigVersion',
        'SmartScreen',
        'Census_InternalBatteryType'
    ]
    
    case_treatment = [
        'SmartScreen',
        'Census_ChassisTypeName',
        'Census_OSEdition',
        'Census_PowerPlatformRoleName',
        'Census_GenuineStateName'
    ]
    
    print("-- replacing weird characters ...")
    for col in char_treatment:
        if df[col].dtype.name == 'object':
            df[col] = df[col].str.replace(r'&#x(\d\d);', '\1', regex=True)
            df[col] = df[col].str.replace(r'[\x00-\x1f]', '', regex=True)
            
    print("-- lower-casing where appropriate ...")
    for col in case_treatment:
        if df[col].dtype.name == 'object':
            df[col] = df[col].str.lower()
            
    #
    # make strings into categories
    #
    
    categories = [
        'SmartScreen',
        'Census_InternalBatteryType',
        'Census_ChassisTypeName',
        'Census_OSEdition'
    ]
    
    print("-- making categories from strings that needed massaging ...")
    for col in categories:
        df[col] = df[col].astype('category')


        
    #
    # add 'unknown' categories where necessary and replace the NAs
    # ADD COLUMNS NAMES HERE TO HAVE THEIR CATEGORIES AUGMENTED AND NAS FILLED WITH 'unknown'
    #
    
    categories = [
        'PuaMode',
        'SmartScreen',
        'Census_ProcessorClass',
        'Census_PrimaryDiskTypeName',  # ['HDD' 'SSD' 'UNKNOWN' 'Unspecified']
        'Census_InternalBatteryType',
        'Census_OSEdition',
        'Census_PowerPlatformRoleName', # also had 'unknown' as well as Nas
        'Census_GenuineStateName'       # and this one too
        
    ]

    print("-- adding categories ..")
    for col in categories:
        df[col].cat.add_categories(['unknown'], inplace=True)
        df[col].fillna('unknown', inplace=True)
    # add one manually because it needs a special unknown value
    df["OsBuildLab"].cat.add_categories(["0.0.-.-.0-0"], inplace=True)
    df["OsBuildLab"].fillna("0.0.-.-.0-0", inplace=True)
    # and this one already had some 'unknown' values
    df['Census_ChassisTypeName'].fillna('unknown', inplace=True)



    #
    # flag and fill selected NAs
    # ADD COLUMN NAMES HERE IN nafill TO HAVE COLUMNS FLAGGED AND FILLED WITH PROVIDED VALUES
    #
    
    print("-- replacing selected NA values")
    nafill = {
        "RtpStateBitfield":0,
        "DefaultBrowsersIdentifier":0,
        "AVProductStatesIdentifier":0,
        "AVProductsInstalled":0,
        "AVProductsEnabled":0,
        "CityIdentifier":0,
        "OrganizationIdentifier":0,
        "GeoNameIdentifier":0,
        "IsProtected":0,
        "SMode":0,
        "IeVerIdentifier":0,
        "Firewall":0,
        "UacLuaenable":0,
        "Census_OEMNameIdentifier":0,
        "Census_OEMModelIdentifier":0,
        "Census_ProcessorCoreCount":0,
        "Census_ProcessorManufacturerIdentifier":0,
        "Census_ProcessorModelIdentifier":0,
        "Census_PrimaryDiskTotalCapacity":0,
        "Census_SystemVolumeTotalCapacity":0,
        "Census_TotalPhysicalRAM":0,
        "Census_InternalPrimaryDiagonalDisplaySizeInInches":0,
        "Census_InternalPrimaryDisplayResolutionHorizontal":0,
        "Census_InternalPrimaryDisplayResolutionVertical":0,
        "Census_InternalBatteryNumberOfCharges":0,
        "Census_OSInstallLanguageIdentifier":0,
        "Census_IsFlightingInternal":0,
        "Census_IsFlightsDisabled":0,
        "Census_ThresholdOptIn":0,
        "Census_FirmwareManufacturerIdentifier":0,
        "Census_IsWIMBootEnabled":0,
        "Census_IsVirtualDevice":0,
        "Census_IsAlwaysOnAlwaysConnectedCapable":0,
        "Wdft_IsGamer":0,
        "Wdft_RegionIdentifier":0,
        "Census_FirmwareVersionIdentifier":0
    }

    for col in nafill:
        df[col+'_wasna'] = df[col].isna()
    df.fillna(value=nafill, inplace=True)
    
    #
    # then some of these columns can become ints, not floats
    #

    print("-- converting columns to int ...")
    df['RtpStateBitfield'] = df['RtpStateBitfield'].astype(np.uint8)

    #
    # deal with version numbers
    #
    
    print("-- mapping version numbers ...")
    def map_OsVer(df,col):
        df_split = df[col].str.split(".", n=3, expand=True)
        df[col+'_major'] = df_split[0].astype(np.int16)
        df[col+'_minor'] = df_split[1].astype(np.int16)
        df[col+'_build1'] = df_split[2].astype(np.int16)
        df[col+'_build2'] = df_split[3].astype(np.int16)
        # the "combined" column is an attempt at making an orginal out of the four values
        df[col+'_combined'] = 10000.0*df[col+'_major']+100.0*df[col+'_minor']+1.0*(df[col+'_build1'])+(df[col+'_build2'])/1000.0

    # e.g. 10.0.10240.16397
    def map_CensusOSVersion(df,col):
        df_split = df[col].str.split(".", n=3, expand=True)
        df[col+'_major'] = df_split[0].astype(np.int16)
        df[col+'_minor'] = df_split[1].astype(np.int16)
        df[col+'_build1'] = df_split[2].astype(np.int16)
        df[col+'_build2'] = df_split[3].astype(np.int16)
        # the "combined" column is an attempt at making an orginal out of the four values
        df[col+'_combined'] = 1000000.0*df[col+'_major']+df[col+'_minor']*10000+(df[col+'_build1'])+(df[col+'_build2'])/100000.0

    # e.g. 1.235.2586.0
    def map_AvSigVersion(df,col):
        df_split = df[col].str.split(".", n=3, expand=True)
        df[col+'_major'] = df_split[0].astype(np.int16)
        df[col+'_minor'] = df_split[1].astype(np.int16)
        df[col+'_build1'] = df_split[2].astype(np.int16)
        df[col+'_build2'] = df_split[3].astype(np.int16)
        # the "combined" column is an attempt at making an orginal out of the four values
        df[col+'_combined'] = 10000.0*df[col+'_major']+df[col+'_minor']+(df[col+'_build1'])/10000.0+(df[col+'_build2'])/10000000.0


    # e.g. 4.12.17007.18022
    def map_AppVersion(df,col):
        df_split = df[col].str.split(".", n=3, expand=True)
        df[col+'_major'] = df_split[0].astype(np.int16)
        df[col+'_minor'] = df_split[1].astype(np.int16)
        df[col+'_build1'] = df_split[2].astype(np.int16)
        df[col+'_build2'] = df_split[3].astype(np.int16)
        # the "combined" column is an attempt at making an orginal out of the four values
        df[col+'_combined'] = 10000000.0*df[col+'_major']+100000.0 * df[col+'_minor']+1.0*(df[col+'_build1'])+(df[col+'_build2'])/100000.0

    # e.g. 1.1.12902.0
    def map_EngineVersion(df,col):
        df_split = df[col].str.split(".", n=3, expand=True)
        df[col+'_major'] = df_split[0].astype(np.int16)
        df[col+'_minor'] = df_split[1].astype(np.int16)
        df[col+'_build1'] = df_split[2].astype(np.int16)
        df[col+'_build2'] = df_split[3].astype(np.int16)
        # the "combined" column is an attempt at making an orginal out of the four values
        df[col+'_combined'] = 100000000.0*df[col+'_major']+1000000.0*df[col+'_minor']+1.0*(df[col+'_build1'])+(df[col+'_build2'])/10000.0

    def map_OsBuildLab(df, col):
        series = df[col].str.replace('*', '.', regex=False)
        df_split = series.str.split(".", n=4, expand=True)
        df[col+'_major'] = df_split[0].astype(np.int16)
        df[col+'_minor'] = df_split[1].astype(np.int16)
        df[col+'_platform'] = df_split[2].astype('category')
        df[col+'_release'] = df_split[3].astype('category')
        df_build = df_split[4].str.split("-", n=1, expand=True)
        df[col+'_build1'] = df_build[0].astype(np.int32)
        df[col+'_build2'] = df_build[1].astype(np.int32)
        # the "combined" column is an attempt at making an orginal out of the four values
        df[col+'_combined'] = 1000000.0*df['OsBuildLab_major']+10.0*df['OsBuildLab_minor']+df['OsBuildLab_build1']/100000.0+df['OsBuildLab_build2']/10000000000.0
        df_split = None
        df_build = None

    map_EngineVersion(df, "EngineVersion")
    map_AppVersion(df, "AppVersion")
    map_AvSigVersion(df, "AvSigVersion")
    map_CensusOSVersion(df, "Census_OSVersion")
    map_OsVer(df, "OsVer")
    map_OsBuildLab(df, "OsBuildLab")    
    
    gc.collect()
    
    return df


In [5]:
class Encoder:
    def __init__(self, df): #in_file):
        self.df = df #pd.read_csv(in_file, dtype=dtypes)
        #print("Completed read operation for", in_file)
        #self.reduce_mem()
        gc.collect()
        self.make_subsets(self.df)
        self.encode_it()
        self.transform_df(self.df, self.nominal_cols)
        self.std_norm()
        
    
    def reduce_mem(self, verbose=True):
        start_mem = self.df.memory_usage().sum() / 1024**2
        print('-- attempting to reduce memory. memory usage of dataframe is {:.2f} MB'.format(start_mem))

        for col in self.df.columns:
            col_type = self.df[col].dtype

            if col_type != object and str(col_type) != 'category':
                c_min = self.df[col].min()
                c_max = self.df[col].max()
                if str(col_type)[:3] == 'int':
                    if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                        self.df[col] = self.df[col].astype(np.int8)
                    elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                        self.df[col] = self.df[col].astype(np.int16)
                    elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                        self.df[col] = self.df[col].astype(np.int32)
                    elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                        self.df[col] = self.df[col].astype(np.int64)  
                # leave floats alone because the downcasting is messing up our mapped values
                #else:
                #    if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                #        df[col] = df[col].astype(np.float16)
                #    elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                #        df[col] = df[col].astype(np.float32)
                #    else:
                #        df[col] = df[col].astype(np.float64)
                else:
                    if str(col_type)[:5] != 'float':
                        self.df[col] = self.df[col].astype('category')

        end_mem = self.df.memory_usage().sum() / 1024**2
        print('-- memory usage after optimization is: {:.2f} MB'.format(end_mem))
        print('-- decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

        return self.df

    def make_subsets(self, df):
        print("-- making subsets ...")
        numerics = ['int8', 'int16', 'int32', 'int64', 'float16', 'float32', 'float64']
        self.numeric_cols = [c for c,v in self.df.dtypes.items() if v in numerics and c in self.df.columns]
        self.nominal_cols = [c for c in self.df.columns if (c not in self.numeric_cols)]
#         # Andrew - still need to fix this
#         self.nominal_cols.remove('SmartScreen')
        self.binary_cols = [c for c in self.df.columns if (self.df[c].nunique() == 2 and c not in self.nominal_cols)]
        self.unary_cols = [c for c in self.df.columns if (self.df[c].nunique() == 1 and c not in self.nominal_cols)]
        if "HasDetections" in df.columns:
            self.labels = df["HasDetections"].values
        print("-- subsets are complete")
        return

    def transform_df(self, in_df, nominal_cols):
        print("-- datatype transformation ...")
        le = preprocessing.LabelEncoder()
        tmp_df = in_df[nominal_cols].apply(le.fit_transform)
        for c in in_df.loc[:, in_df.dtypes == np.int8].columns:
            tmp_df[c] = in_df[c]
        for c in in_df.loc[:, in_df.dtypes == np.int16].columns:
            tmp_df[c] = in_df[c]
        for c in in_df.loc[:, in_df.dtypes == np.int32].columns:
            tmp_df[c] = in_df[c]
        for c in in_df.loc[:, in_df.dtypes == np.int64].columns:
            tmp_df[c] = in_df[c]
        for c in in_df.loc[:, in_df.dtypes == np.float16].columns:
            tmp_df[c] = in_df[c]
        for c in in_df.loc[:, in_df.dtypes == np.float32].columns:
            tmp_df[c] = in_df[c]
        for c in in_df.loc[:, in_df.dtypes == np.float64].columns:
            tmp_df[c] = in_df[c]
        for c in in_df[in_df.select_dtypes(bool).columns]:
            tmp_df[c] = in_df[c]
        self.df = tmp_df
        print("-- completed transforming dtypes")
        return

    def std_norm(self):
        print("-- scaling ...")
        col_to_std = ['AVProductStatesIdentifier',
                      'CountryIdentifier',
                      'CityIdentifier',
                      'GeoNameIdentifier',
                      'LocaleEnglishNameIdentifier',
                      'OsBuild',
                      'IeVerIdentifier',
                      'Census_OEMNameIdentifier',
                      'Census_OEMModelIdentifier',
                      'Census_ProcessorCoreCount',
                      'Census_ProcessorModelIdentifier',
                      'Census_PrimaryDiskTotalCapacity',
                      'Census_SystemVolumeTotalCapacity',
                      'Census_TotalPhysicalRAM',
                      'Census_InternalPrimaryDiagonalDisplaySizeInInches',
                      'Census_InternalPrimaryDisplayResolutionHorizontal',
                      'Census_InternalPrimaryDisplayResolutionVertical',
                      'Census_InternalBatteryNumberOfCharges',
                      'Census_OSBuildNumber',
                      'Census_OSInstallLanguageIdentifier',
                      'Census_OSUILocaleIdentifier',
                      'Census_FirmwareManufacturerIdentifier',
                      'Census_FirmwareVersionIdentifier',
                      'Wdft_RegionIdentifier',
                      'OsBuildLab_major',
                      'OsBuildLab_minor',
                      'OsBuildLab_platform',
                      'OsBuildLab_release',
                      'OsBuildLab_build2']
        scaled_features = self.df.copy()
        features = scaled_features[col_to_std]
        scaler = StandardScaler().fit(features.values)
        features = scaler.transform(features.values)
        scaled_features[col_to_std] = features
        self.df = scaled_features
        print("-- completed standardization and normalization")
        return
    
    def encode_it(self):
        print("-- label encoding ...")
        le = preprocessing.LabelEncoder()
        for n in self.nominal_cols:
            #print(" ...",n)
            self.df[n] = le.fit_transform(self.df[n])
        print("-- completed label encoding")
        return
    
    def get_encoded_data(self):
        return self.df
    

In [6]:
# improved function to create a mini set from the supplied criteria (n, features, labels)
def generate_mini(n, features, labels):
    sample_size = n / features.shape[0]
    reserved_size = 1-sample_size
    X_mini, X_rest, y_mini, y_rest = train_test_split(features, labels, stratify=labels, test_size=reserved_size, random_state=0)
    return X_mini, X_rest, y_mini, y_rest

In [8]:
# function to create stratified train, dev and validate sets from supplied ratios 
def generate_train_dev_validate_sets(train_ratio, test_ratio, features, labels):
    reserved_size = 1-train_ratio
    X_train, X_rest, y_train, y_rest =  \
        train_test_split(features, labels, stratify=labels, test_size=reserved_size, random_state=0)
    reserved_size = 1 - (test_ratio / reserved_size)
    X_dev, X_validate, y_dev, y_validate = \
        train_test_split(X_rest, y_rest, stratify=y_rest, test_size=reserved_size, random_state=0)
    return X_train, X_dev, X_validate, y_train, y_dev, y_validate

In [9]:
#
# function for saving frames
#

def save_encoded_files(df, name):
    
    # save the file
    df.to_csv("data/"+name+"_encoded.csv", index=False)
    

In [10]:
#
#
# below here is diagnostic and test functionality, invoke as needed
#
#

In [11]:
# getting our dtypes for loading the file back when we need it
def dump_final_types(df):
    for dtype in df.dtypes.items():
        print("'{:} '{:}',".format((dtype[0] + "':").ljust(54), dtype[1]))

In [12]:
# Unit testing the version mapping routines
def ver_func_unit_test():
    unitTestData = {'EngineVersion':['1.1.12902.0', '1.1.13000.0', '1.1.13103.0', '1.1.13202.0', '1.1.13303.0',
                                     '1.1.13407.0', '1.1.13504.0', '1.1.13601.0', '1.1.13701.0', '1.1.13804.0',
                                     '1.1.13903.0', '1.1.14003.0', '1.1.14104.0', '1.1.14202.0', '1.1.14303.0',
                                     '1.1.14305.0', '1.1.14306.0', '1.1.14405.2', '1.1.14500.5', '1.1.14600.4',
                                     '1.1.14700.5', '1.1.14800.1', '1.1.14800.3', '1.1.14901.3', '1.1.14901.4',
                                     '1.1.15000.1', '1.1.15000.2', '1.1.15100.1', '1.1.15200.1', '1.1.15300.5',
                                     '1.1.15300.6', '1.1.12805.0', '1.1.13704.0', '1.1.14700.3', '1.1.14700.4',
                                     '1.1.12101.0', '1.1.13802.0', '1.1.11502.0', '1.1.11701.0', '1.1.14002.0',
                                     '1.1.14102.0', '1.1.14201.0', '1.1.14500.2', '1.1.13102.0', '1.1.12400.0',
                                     '1.1.12603.0', '1.1.14103.0', '1.1.13902.0', '1.1.13504.0', '1.1.9700.0'],
                       'AppVersion':['4.10.14393.0', '4.10.14393.1066', '4.10.14393.1198', '4.10.14393.1593',
                                     '4.10.14393.1613', '4.10.14393.1794', '4.10.14393.2273', '4.10.14393.953',
                                     '4.10.205.0', '4.10.209.0', '4.11.15063.0', '4.11.15063.1155',
                                     '4.11.15063.447', '4.12.16299.15', '4.12.17007.17123', '4.12.17007.18011',
                                     '4.12.17007.18022', '4.13.17134.1', '4.13.17134.112', '4.13.17134.191',
                                     '4.13.17134.228', '4.14.17613.18038', '4.14.17613.18039', '4.14.17639.18041',
                                     '4.16.17656.18052', '4.17.17686.1003', '4.18.1806.18062', '4.18.1807.18075',
                                     '4.18.1807.20063', '4.18.1809.2', '4.5.218.0', '4.6.305.0', '4.8.10240.16384',
                                     '4.8.10240.17071', '4.8.10240.17113', '4.8.10240.17202', '4.8.10240.17394',
                                     '4.8.10240.17443', '4.8.10240.17609', '4.8.10240.17861', '4.8.10240.17889',
                                     '4.8.10240.17914', '4.8.207.0', '4.9.10586.0', '4.9.10586.1045',
                                     '4.9.10586.1106', '4.9.10586.494', '4.9.10586.589', '4.9.218.0', '4.12.17007.17121'],
                     'AvSigVersion':['1.225.1338.0', '1.225.1947.0', '1.225.394.0', '1.227.2357.0', '1.227.2846.0',
                                     '1.227.2939.0', '1.229.1557.0', '1.229.1669.0', '1.229.1848.0',
                                     '1.231.1226.0', '1.231.1362.0', '1.233.1808.0', '1.235.1725.0',
                                     '1.235.2115.0', '1.235.2586.0', '1.235.428.0', '1.237.0.0', '1.237.1259.0',
                                     '1.237.1572.0', '1.237.1702.0', '1.237.782.0', '1.237.787.0', '1.239.1512.0',
                                     '1.239.230.0', '1.239.37.0', '1.239.460.0', '1.239.956.0', '1.241.296.0',
                                     '1.241.643.0', '1.245.1013.0', '1.245.1110.0', '1.245.449.0', '1.245.931.0',
                                     '1.245.977.0', '1.247.347.0', '1.247.641.0', '1.249.1361.0', '1.249.281.0',
                                     '1.249.557.0', '1.249.713.0', '1.249.894.0', '1.251.1027.0', '1.251.1500.0',
                                     '1.251.170.0', '1.251.359.0', '1.251.42.0', '1.251.505.0', '1.251.878.0',
                                     '0.0.0.0', '1.2173.1144.0'],
                 'Census_OSVersion':['10.0.10240.16384', '10.0.10240.16397', '10.0.10240.16405',
                                     '10.0.10240.16445', '10.0.10240.16463', '10.0.10240.16487',
                                     '10.0.10240.16644', '10.0.10240.17071', '10.0.10240.17202',
                                     '10.0.10240.17394', '10.0.10240.17443', '10.0.10240.17709',
                                     '10.0.10240.17861', '10.0.10240.17889', '10.0.10240.17914', '10.0.10586.0',
                                     '10.0.10586.1007', '10.0.10586.104', '10.0.10586.1045', '10.0.10586.1106',
                                     '10.0.10586.1176', '10.0.10586.122', '10.0.10586.14', '10.0.10586.164',
                                     '10.0.10586.17', '10.0.10586.218', '10.0.10586.29', '10.0.10586.3',
                                     '10.0.10586.318', '10.0.10586.36', '10.0.10586.420', '10.0.10586.494',
                                     '10.0.10586.545', '10.0.10586.589', '10.0.10586.63', '10.0.10586.633',
                                     '10.0.10586.679', '10.0.10586.71', '10.0.10586.713', '10.0.10586.753',
                                     '10.0.10586.839', '10.0.10586.873', '10.0.10586.916', '10.0.10586.962',
                                     '10.0.14393.0', '10.0.14393.105', '10.0.14393.1066', '10.0.14393.1198',
                                     '10.0.17134.1', '10.0.10240.16724'],
                            'OsVer':['10.0.0.0', '10.0.0.1', '10.0.0.112', '10.0.0.2', '10.0.0.22', '10.0.0.250',
                                     '10.0.0.3', '10.0.0.80', '10.0.0.96', '10.0.1.0', '10.0.1.144', '10.0.1.244',
                                     '10.0.1.44', '10.0.153.153', '10.0.16.0', '10.0.16.36', '10.0.19.80',
                                     '10.0.2.0', '10.0.2.80', '10.0.2.86', '10.0.21.0', '10.0.23.0', '10.0.26.128',
                                     '10.0.3.0', '10.0.3.232', '10.0.3.80', '10.0.32.0', '10.0.32.72', '10.0.4.0',
                                     '10.0.4.80', '10.0.48.0', '10.0.5.0', '10.0.5.117', '10.0.5.18', '10.0.6.0',
                                     '10.0.64.150', '10.0.7.0', '10.0.7.101', '10.0.7.80', '10.0.72.0', '10.0.8.0',
                                     '10.0.80.0', '6.1.0.0', '6.1.0.112', '6.1.0.128', '6.1.1.0', '6.1.16.36',
                                     '6.1.2.0', '6.1.3.0', '6.1.4.0'],
                       'OsBuildLab':['10240.16384.amd64fre.th1.150709-1700',
                                     '10240.16393.amd64fre.th1_st1.150717-1719',
                                     '10240.16393.x86fre.th1_st1.150717-1719',
                                     '10240.16431.amd64fre.th1.150810-2333',
                                     '10240.16431.x86fre.th1.150810-2333',
                                     '10240.16463.amd64fre.th1.150819-1946',
                                     '10240.17071.amd64fre.th1.160802-1852',
                                     '10240.17113.amd64fre.th1.160906-1755',
                                     '10240.17202.amd64fre.th1_st1.161118-1836',
                                     '10240.17394.amd64fre.th1_st1.170427-1347',
                                     '10240.17443.amd64fre.th1.170602-2340',
                                     '10240.17443.x86fre.th1.170602-2340', '10240.17709.x86fre.th1.171130-0900',
                                     '10240.17861.amd64fre.th1.180427-1806',
                                     '10240.17889.amd64fre.th1_st1.180529-1823',
                                     '10240.17914.amd64fre.th1.180627-1911',
                                     '10586.0.amd64fre.th2_release.151029-1700',
                                     '10586.0.x86fre.th2_release.151029-1700',
                                     '10586.1007.amd64fre.th2_release.170706-2002',
                                     '10586.103.amd64fre.th2_release.160126-1819',
                                     '10586.1045.amd64fre.th2_release.170728-1941',
                                     '10586.1045.x86fre.th2_release.170728-1941',
                                     '10586.11.amd64fre.th2_release.151112-1900',
                                     '10586.1106.amd64fre.th2_release.170904-1742',
                                     '10586.1176.amd64fre.th2_release_sec.170913-1848',
                                     '10586.1176.x86fre.th2_release_sec.170913-1848',
                                     '10586.122.amd64fre.th2_release_inmarket.160222-1549',
                                     '10586.162.amd64fre.th2_release_sec.160223-1728',
                                     '10586.162.x86fre.th2_release_sec.160223-1728',
                                     '10586.17.amd64fre.th2_release.151121-2308',
                                     '10586.212.amd64fre.th2_release_sec.160328-1908',
                                     '10586.212.x86fre.th2_release_sec.160328-1908',
                                     '10586.3.amd64fre.th2_release_sec.151104-1948',
                                     '10586.306.amd64fre.th2_release_sec.160422-1850',
                                     '10586.420.amd64fre.th2_release_sec.160527-1834',
                                     '10586.420.x86fre.th2_release_sec.160527-1834',
                                     '10586.494.x86fre.th2_release_sec.160630-1736',
                                     '10586.545.amd64fre.th2_release.160802-1857',
                                     '10586.589.amd64fre.th2_release.160906-1759',
                                     '10586.63.amd64fre.th2_release.160104-1513',
                                     '10586.633.amd64fre.th2_release.161004-1602',
                                     '10586.672.amd64fre.th2_release_sec.161024-1825',
                                     '10586.672.x86fre.th2_release_sec.161024-1825',
                                     '10586.839.amd64fre.th2_release.170303-1605',
                                     '10586.839.x86fre.th2_release.170303-1605',
                                     '10586.916.amd64fre.th2_release_sec.170427-1350',
                                     '10586.962.amd64fre.th2_release.170602-2241',
                                     '10586.962.x86fre.th2_release.170602-2241',
                                     '0.0.-.-.0-0',
                                     '18238.1000.amd64fre.rs_onecore_sigma_grfx_dev.180911-1700']
                   }

    unitTestDf = pd.DataFrame(unitTestData) 

    # method to test (copied from the notebook)
    # 10.0.153.153  1000.15315299999997


    pd.options.display.float_format = '{:.14f}'.format
    set_of_cols =['EngineVersion_combined','EngineVersion','AppVersion_combined', 'AppVersion', 'AvSigVersion_combined', 'AvSigVersion', 'Census_OSVersion_combined', 'Census_OSVersion', 'OsVer_combined', 'OsVer', 'OsBuildLab_combined', 'OsBuildLab']
    return unitTestDf[set_of_cols]




In [13]:
#
# main code part one: train dataset
#

start = time.time()

if debug:
    print("using mini_initial.csv dataset")
    filename = "mini_initial"
else:        
    print("using big dataset")
    filename = "train"
    
#
# initial load and casting to desired types
#

df = load_data(filename)
print("done loading train data")
print("total rows in set:", len(df))
print("seconds elapsed:", time.time()-start)

#
# make mini_initial, before cleaning and dropping, if necessary (to debug wrangler code)
#
   
if not debug:
    print("creating mini_initial.csv for debug purposes ...")
    df2 = df.sample(100000, random_state=123)
    df2.to_csv("data/mini_initial.csv", index=False)
    print("created mini_initial.csv data")
    print("seconds elapsed:", time.time()-start)
    
#
# main work is done here
#
    
print("cleaning train.csv or mini_initial.csv data ...")
df = clean_data(df)
print("encoding train data ...")
df = Encoder(df).get_encoded_data()
print("done cleaning and encoding train data")
print("seconds elapsed:", time.time()-start)

print("generating smaller stratified sets ...")

# generate stratified data sets using supplied ratios to create the sets
df_train, df_dev, df_validate, train_labels, dev_labels, validate_labels = \
    generate_train_dev_validate_sets(.7, .15, df, df['HasDetections'])

df=[] # release df from memory


using big dataset
done loading train data
total rows in set: 8921483
seconds elapsed: 91.0527777671814
creating mini_initial.csv for debug purposes ...
created mini_initial.csv data
seconds elapsed: 97.91017484664917
cleaning train.csv or mini_initial.csv data ...
-- replacing weird characters ...
-- lower-casing where appropriate ...
-- making categories from strings that needed massaging ...
-- adding categories ..
-- replacing selected NA values
-- converting columns to int ...
-- mapping version numbers ...
encoding train data ...
-- making subsets ...
-- subsets are complete
-- label encoding ...
-- completed label encoding
-- datatype transformation ...
-- completed transforming dtypes
-- scaling ...
-- completed standardization and normalization
done cleaning and encoding train data
seconds elapsed: 551.5067896842957
generating smaller stratified sets ...


In [14]:
#
# create mini_ files for model development
#
mini_ratio = .05

mini_train, remaining, mini_train_labels, remaining_labels = \
    generate_mini(df_train.shape[0]*mini_ratio, df_train, df_train['HasDetections'])

mini_dev, remaining, mini_dev_labels, remaining_labels = \
    generate_mini(df_dev.shape[0]*mini_ratio, df_dev, df_dev['HasDetections'])

mini_validate, remaining, mini_validate_labels, remaining_labels = \
    generate_mini(df_validate.shape[0]*mini_ratio, df_validate, df_validate['HasDetections'])


shape of mini_train: (312251, 151)
shape of mini_dev: (66911, 151)
379162
shape of mini_test: (66911, 151)
379162
done making mini sets
seconds elapsed: 679.0455458164215


In [40]:
print("shape of mini_train:",mini_train.shape)
print("shape of mini_dev:",mini_dev.shape)
print("shape of mini_validate:",mini_validate.shape)
print("\nchecking symmetric difference between mini_dev and mini_train:", len(set(mini_train['MachineIdentifier']).symmetric_difference(mini_dev['MachineIdentifier'])))
print("-- total unique machine ids after recombining both sets:", pd.concat([mini_train['MachineIdentifier'], mini_dev['MachineIdentifier']]).nunique())
print("-- original length of mini_dev + length of mini_train:", mini_dev.shape[0] + mini_train.shape[0])
print("\nchecking symmetric difference between mini_validate and mini_train:", len(set(mini_train['MachineIdentifier']).symmetric_difference(mini_validate['MachineIdentifier'])))
print("-- total unique machine ids after recombining both sets:", pd.concat([mini_train['MachineIdentifier'], mini_validate['MachineIdentifier']]).nunique())
print("-- original length of mini_validate + length of mini_train:", mini_validate.shape[0] + mini_train.shape[0])
print("\nchecking symmetric difference between mini_dev and mini_validate:", len(set(mini_dev['MachineIdentifier']).symmetric_difference(mini_validate['MachineIdentifier'])))
print("-- total unique machine ids after recombining both sets:", pd.concat([mini_dev['MachineIdentifier'], mini_validate['MachineIdentifier']]).nunique())
print("-- original length of mini_dev + length of mini_validate:", mini_validate.shape[0] + mini_dev.shape[0])
print("\ndone making mini sets")
print("seconds elapsed:", time.time()-start)

shape of mini_train: (312251, 151)
shape of mini_dev: (66911, 151)
shape of mini_validate: (66911, 151)

checking symmetric difference between mini_dev and mini_train: 379162
-- total unique machine ids after recombining both sets: 379162
-- original length of mini_dev + length of mini_train: 379162

checking symmetric difference between mini_validate and mini_train: 379162
-- total unique machine ids after recombining both sets: 379162
-- original length of mini_validate + length of mini_train: 379162

checking symmetric difference between mini_dev and mini_validate: 133822
-- total unique machine ids after recombining both sets: 133822
-- original length of mini_dev + length of mini_validate: 133822

done making mini sets
seconds elapsed: 1928.0604047775269


In [42]:
#
# save encoded mini files
# save function will add "data/" and "_encoded.csv"
#

if save_data:
    print("saving mini_train_encoded ...")
    save_encoded_files(mini_train, "mini_train")
    print("saving mini_dev_encoded ...")
    save_encoded_files(mini_dev, "mini_dev")
    print("saving mini_validate_encoded ...")
    save_encoded_files(mini_validate, "mini_validate")
    print("done saving mini files")
    print("seconds elapsed:", time.time()-start)


saving mini_train_encoded ...
saving mini_dev_encoded ...
saving mini_validate_encoded ...
done saving mini files
seconds elapsed: 2154.518833875656


In [ ]:
#
# save encoded full files
# save function will add "data/" and "_encoded.csv"
#

if save_data:
    print("saving train_encoded ...")
    save_encoded_files(df_train, "train")
    print("done saving encoded train file")

    print("saving dev_encoded ...")
    save_encoded_files(df_dev, "dev")
    print("done saving encoded dev file")

    print("saving validate_encoded ...")
    save_encoded_files(df_validate, "validate")
    print("done saving encoded validate file")



saving train_encoded ...


In [ ]:
#
# part two: large test dataset
#
    
if not debug:
    print("loading test data ...")
    df_test = load_data("test")
    print("done loading test data")
    print("seconds elapsed:", time.time()-start)
    print("total rows in test set:", len(df_test))
    print("cleaning large test data ...")
    df_test = clean_data(df_test)
    print("encoding large test data ...")
    df_test = Encoder(df_test).get_encoded_data()
    print("done cleaning and encoding large test data")
# commenting this out so it doesn't happen
# since this small test set is not a subset of the larger test set
# and having it around maybe causing confusion
#else:
#    print("faking large test data ...")
#    df_test_large = df_test_small.copy()   
print("seconds elapsed:", time.time()-start)



In [ ]:
if save_data:
    print("saving encoded large test data ...")
    save_encoded_files(df_test, "test")
    print("done saving encoded files")
    print("seconds elapsed:", time.time()-start)

#
# at this point, the sets are in the following dataframes:
#
# df_train, df_dev, df_test_small: 
#    main large sets, all derived from train.csv (unles debugging)
# mini_train, mini_dev, mini_test: 
#    really small files for development, all derived from train.csv (unless debugging)
# df_test_large: 
#    test.csv dataset (or same as df_test_small, if debugging)
#
     
print("all done.")

In [ ]:
dump_final_types(df_train)